In [3]:
from bs4 import BeautifulSoup
import requests

In [115]:
n_of_pages=4
baselink = "https://www.aekszerek.hu/arany/noi-arany-nyaklancok.html?lis="
urls_for_items = [] # e.g."https://www.aekszerek.hu/ezust/nyaklancok/noi/ezust-nyaklanc-sziv-p156367.html",

for page in range(n_of_pages):
    soupbase = BeautifulSoup(requests.get(baselink + str(page)).content, "html.parser")

    items_on_page = soupbase.find_all('div', class_="item_img")
    urls_for_items_on_this_page = ["https://www.aekszerek.hu/" + item.a['href'] for item in items_on_page]


    urls_for_items = urls_for_items + urls_for_items_on_this_page

print(f"{len(urls_for_items)} item-url is found after going through {n_of_pages} pages")

317 item-url is found after going through 4 pages


In [116]:
def get_details_of_items(itemurl):
    pass

In [117]:
itemurl = urls_for_items[0]

In [118]:
soup = BeautifulSoup(requests.get(itemurl).content, "html.parser")

In [119]:
soup.title.text

'Nyaklánc szívvel iGold, nőknek (IZ19400M) | aEkszerek.hu'

In [120]:
# soup.find('div', class_="price")
soup.find('span', id="pricevalue")

<span class="now-price" id="pricevalue">
56 510 Ft
</span>

In [124]:
details_in_string = soup.find('div', class_='col-md-8 col-sm-12 col-xs-12 margin-bottom10 toggle-content').text
details_in_string

' \nAnyag: Arany 14K,\nNyaklánc kapocs: rugós kapocs,\nMotívum: Szívecskék,\nCélcsoport: Női,\nÉkszer súlya: 1.65 gramm,\nTisztaság: 0,585,\nArany színe: Sárga arany,\nHossz: Válassza ki a méretet,\nSzélesség: 12 mm,\nMagasság: 12,\nÁllapot: Rendszerint kézbesítve: 2 - 6 munkanapon belül,\nKő: Cirkon,\n Nyaklánc 14 karátos sárga aranyból. Finom láncból és faragott szív alakú medálból áll, csillogó cirkóniákkal díszítve. Kényeztesse magát ezzel a nyaklánccal, amely gyönyörűen díszíti majd a nyakkivágását. '

In [127]:
details_in_string = details_in_string.replace('\n', '').replace(' ','')